In [1]:
from pathlib import Path
import sys

# notebooks/ から見てプロジェクトルートを追加
ROOT = Path.cwd().resolve().parents[0]  # notebooks の1つ上 = ~/work/keiba-ai
sys.path.append(str(ROOT))

In [2]:
from src.data.loaders.result_loader import load_results
from src.data.loaders.horse_loader import load_horse_results
from src.data.loaders.race_info_loader import load_race_info

df_result = load_results("../data/rawdf/result/result_*.csv")
df_horse = load_horse_results("../data/rawdf/horse/*.csv")
df_race_info = load_race_info("../data/rawdf/race_info/*.csv")

/home/naga2/work/keiba-ai/src/data/loaders/horse_loader.py:13: DtypeWarning: Columns (0: 着順) have mixed types. Specify dtype option on import or set low_memory=False.
  raw = pd.read_csv(p, sep=",")


In [3]:
from src.data.pipelines.build_train_table import build_train_table

df_train = build_train_table(df_result, df_race_info, df_horse)
df_train

,race_id,horse_id,race_date,place,race_type,around,course_len,weather,ground_state,race_class,...,impost,weight,weight_diff,jockey_id,trainer_id,owner_id,popularity,tansho_odds,hr_rank_mean_3,rank
0,202101010101,2018104780,2021-06-12,0,0,0.0,1200,2,0,-3.0,...,56.0,436,0,1144,1136,414030,6,10.4,5.000000,15
1,202101010101,2018100570,2021-06-12,0,0,0.0,1200,2,0,-3.0,...,54.0,456,4,666,1039,728008,1,3.7,5.666667,6
2,202101010101,2018101518,2021-06-12,0,0,0.0,1200,2,0,-3.0,...,52.0,388,-20,1173,1093,942006,16,275.3,13.000000,10
3,202101010101,2018101447,2021-06-12,0,0,0.0,1200,2,0,-3.0,...,56.0,498,-4,1091,1076,546800,11,57.7,11.000000,12
4,202101010101,2018101539,2021-06-12,0,0,0.0,1200,2,0,-3.0,...,54.0,456,8,1015,417,163002,4,7.5,3.666667,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235833,202510020812,2022105755,2025-07-20,9,0,0.0,1200,0,0,2.0,...,53.0,452,4,1187,1135,520031,3,7.2,6.000000,5
235834,202510020812,2017101767,2025-07-20,9,0,0.0,1200,0,0,2.0,...,58.0,480,8,1125,1120,892002,16,111.0,7.666667,12
235835,202510020812,2020106727,2025-07-20,9,0,0.0,1200,0,0,2.0,...,58.0,544,0,1213,1205,798033,5,18.1,8.333333,8
235836,202510020812,2021103312,2025-07-20,9,0,0.0,1200,0,0,2.0,...,58.0,496,-8,1154,1157,170800,8,29.0,4.000000,3


In [4]:
import pandas as pd
import numpy as np

df = df_train.copy()
df.info()


<class 'pandas.DataFrame'>
RangeIndex: 235838 entries, 0 to 235837
Data columns (total 24 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   race_id         235838 non-null  int64         
 1   horse_id        235838 non-null  int64         
 2   race_date       235838 non-null  datetime64[us]
 3   place           235838 non-null  int64         
 4   race_type       235838 non-null  int64         
 5   around          229062 non-null  float64       
 6   course_len      235838 non-null  int64         
 7   weather         235838 non-null  int64         
 8   ground_state    235838 non-null  int64         
 9   race_class      235838 non-null  float64       
 10  wakuban         235838 non-null  Int64         
 11  umaban          235838 non-null  Int64         
 12  sex             235838 non-null  int64         
 13  age             235838 non-null  Int64         
 14  impost          235838 non-null  float64       

In [5]:
import pandas as pd
import numpy as np

# 目的変数
target = "rank"

# 欠損率
na_rate = (df.isna().mean().sort_values(ascending=False) * 100).round(2)
display(na_rate.head(30))

# 数値/カテゴリ列
num_cols = df.select_dtypes(include=["number"]).columns.tolist()
cat_cols = [c for c in df.columns if c not in num_cols and c != target]
print("num:", len(num_cols), "cat:", len(cat_cols))

# 基本統計
display(df[num_cols].describe().T)


hr_rank_mean_3    10.02
around             2.87
race_date          0.00
race_id            0.00
place              0.00
race_type          0.00
course_len         0.00
horse_id           0.00
weather            0.00
ground_state       0.00
wakuban            0.00
race_class         0.00
sex                0.00
age                0.00
impost             0.00
umaban             0.00
weight             0.00
weight_diff        0.00
trainer_id         0.00
jockey_id          0.00
owner_id           0.00
popularity         0.00
tansho_odds        0.00
rank               0.00
dtype: float64

num: 22 cat: 2


,count,mean,std,min,25%,50%,75%,max
race_id,235838.0,202306137160.765747,141691089.976659,202101010101.0,202204040511.0,202306040201.0,202408040203.0,202510020812.0
horse_id,235838.0,2019493803.798892,1950048.838047,2010101667.0,2018104665.0,2019106856.0,2021103214.0,2023110106.0
place,235838.0,5.12964,2.447431,0.0,4.0,5.0,7.0,9.0
race_type,235838.0,0.559032,0.551344,0.0,0.0,1.0,1.0,2.0
around,229062.0,0.395452,0.505205,0.0,0.0,0.0,1.0,2.0
course_len,235838.0,1655.450818,416.465703,1000.0,1400.0,1600.0,1800.0,4260.0
weather,235838.0,0.500916,0.672533,0.0,0.0,0.0,1.0,3.0
ground_state,235838.0,0.446387,0.794068,0.0,0.0,0.0,1.0,3.0
race_class,235838.0,-1.025992,2.026539,-4.0,-3.0,-1.0,0.0,5.0
wakuban,235838.0,4.76249,2.270728,1.0,3.0,5.0,7.0,8.0


In [6]:
# around 欠損の race_id をユニークに取得
around_na_race_ids = (
    df.loc[df["course_len"].isna(), "race_id"]
      .drop_duplicates()
      .sort_values()
      .astype("int64")  # race_idがintなら
      .tolist()
)

print("around欠損の race_id 数:", len(around_na_race_ids))
print("先頭50件:", around_na_race_ids[:50])



around欠損の race_id 数: 0
先頭50件: []
